In [1]:
# 필수 패키지 설치
!pip install wget
!apt-get install -y sox libsndfile1 ffmpeg
# sox: 오디오 파일 변환/처리
# libsndfile1: 다양한 오디오 포맷 지원
# ffmpeg: 멀티미디어 처리

!pip install text-unidecode
# 텍스트 정규화

!pip install matplotlib>=3.3.2
# 시각화 용도

# NeMo 설치 (main 브랜치 사용, causal-conv1d 의존성 제거)
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

# TorchAudio 설치 (NeMo에서 권장)
!pip install torchaudio>=0.10.0 -f https://download.pytorch.org/whl/torch_stable.html


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsndfile1 is already the newest version (1.0.31-2ubuntu0.2).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
sox is already the newest version (14.4.2+git20190427-2+deb11u2ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
DEPRECATION: git+https://github.com/NVIDIA/NeMo.git@main#egg=nemo_toolkit[all] contains an egg fragment with a non-PEP 508 name. pip 25.3 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/13157
  Cloning https://github.com/NVIDIA/NeMo.git (to revision main) to /tmp/pip-install-8dvjky18/nemo-toolkit_020fa2eabbd047dca5a6cb139c559c36
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA/NeMo.git /tmp/pip-install-8dvjky18/nemo-toolkit_020fa2eabbd047dca5a6cb139c559c36
  R

In [2]:
import os
from pytorch_lightning import Trainer
import nemo.collections.asr as nemo_asr
from nemo.collections.asr.models import EncDecCTCModel


[NeMo W 2025-09-15 08:46:36 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
      m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
    
[NeMo W 2025-09-15 08:46:36 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
      m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
    
[NeMo W 2025-09-15 08:46:36 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
      elif re.match('(flt)p?( \(default\))?$', token):
    
[NeMo W 2025-09-15 08:46:36 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
      elif re.match('(dbl)p?( \(default\))?$', token):
    


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# 사용 가능한 한국어 ASR 모델 확인

from nemo.collections.asr.models import EncDecCTCModel

EncDecCTCModel.list_available_models()

[PretrainedModelInfo(
 	pretrained_model_name=QuartzNet15x5Base-En,
 	description=QuartzNet15x5 model trained on six datasets: LibriSpeech, Mozilla Common Voice                 (validated clips from en_1488h_2019-12-10), WSJ, Fisher, Switchboard, and NSC Singapore English.                     It was trained with Apex/Amp optimization level O1 for 600 epochs. The model achieves a WER of                     3.79% on LibriSpeech dev-clean, and a WER of 10.05% on dev-other. Please visit                         https://ngc.nvidia.com/catalog/models/nvidia:nemospeechmodels for further details.,
 	location=https://api.ngc.nvidia.com/v2/models/nvidia/nemospeechmodels/versions/1.0.0a5/files/QuartzNet15x5Base-En.nemo
 ),
 PretrainedModelInfo(
 	pretrained_model_name=stt_en_quartznet15x5,
 	description=For details about this model, please visit https://ngc.nvidia.com/catalog/models/nvidia:nemo:stt_en_quartznet15x5,
 	location=https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_quartznet15x5/v

In [7]:
MANIFEST_PATH = "/content/drive/MyDrive/NeMo_Korean_ASR/data/processed/kss/kss_manifest.csv"
CHECKPOINT_DIR = "/content/drive/MyDrive/NeMo_Korean_ASR/checkpoints"

os.makedirs(CHECKPOINT_DIR, exist_ok=True)


In [8]:
asr_model = EncDecCTCModel.from_pretrained(model_name="QuartzNet15x5Base-En")

train_manifest = MANIFEST_PATH  # KSS 전체를 train으로 사용
val_manifest = MANIFEST_PATH    # 작은 데이터셋이라 validation도 동일하게 사용

[NeMo I 2025-09-15 09:00:12 nemo_logging:393] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemospeechmodels/versions/1.0.0a5/files/QuartzNet15x5Base-En.nemo to /root/.cache/torch/NeMo/NeMo_2.6.0rc0/QuartzNet15x5Base-En/b69977d31abaf2ab3e8f082866f6dfcb/QuartzNet15x5Base-En.nemo
[NeMo I 2025-09-15 09:00:13 nemo_logging:393] Instantiating model from pre-trained checkpoint
[NeMo I 2025-09-15 09:00:15 nemo_logging:393] PADDING: 16
[NeMo I 2025-09-15 09:00:18 nemo_logging:393] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_2.6.0rc0/QuartzNet15x5Base-En/b69977d31abaf2ab3e8f082866f6dfcb/QuartzNet15x5Base-En.nemo.


In [11]:
train_config = {
    'manifest_filepath': train_manifest,
    'batch_size': 8,
    'shuffle': True
}

val_config = {
    'manifest_filepath': val_manifest,
    'batch_size': 8,
    'shuffle': False
}

asr_model.setup_training_data(train_data_config=train_config)
asr_model.setup_validation_data(val_data_config=val_config)

[NeMo I 2025-09-15 09:01:29 nemo_logging:393] Model level config does not contain `sample_rate`, please explicitly provide `sample_rate` to the dataloaders.
[NeMo I 2025-09-15 09:01:29 nemo_logging:393] Model level config does not contain `labels`, please explicitly provide `labels` to the dataloaders.


[NeMo W 2025-09-15 09:01:29 nemo_logging:405] dataset does not have explicitly defined labels


KeyError: 'sample_rate'

In [ ]:
asr_model.setup_training_data(train_data_config)
asr_model.setup_validation_data(val_data_config)

trainer = Trainer(
    max_epochs=5,  # 테스트용
    accelerator="gpu" if os.environ.get("COLAB_GPU") else "cpu",
)


In [ ]:

trainer.fit(asr_model)

# 모델 저장
OUTPUT_DIR = "/content/drive/MyDrive/NeMo_Korean_ASR/output/kss_finetuned"
os.makedirs(OUTPUT_DIR, exist_ok=True)
asr_model.save_to(OUTPUT_DIR + "/kss_citrinet_finetuned.nemo")

print(f"모델 저장 경로: {OUTPUT_DIR}")